In [1]:
import torch
torch.__version__

'2.0.1'

# 使用PyTorch计算梯度数值
PyTorch的Autograd模块实现了深度学习的算法中的向传播求导数，在张量（Tensor类）上的所有操作，Autograd都能为他们自动提供微分，简化了手动计算导数的复杂过程。

在0.4以前的版本中，Pytorch 使用 Variable 类来自动计算所有的梯度。Variable类主要包含三个属性： data：保存Variable所包含的Tensor；grad：保存data对应的梯度，grad也是个Variable，而不是Tensor，它和data的形状一样；grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度。

从0.4起， Variable 正式合并入Tensor类，通过Variable嵌套实现的自动微分功能已经整合进入了Tensor类中。虽然为了代码的兼容性还是可以使用Variable(tensor)这种方式进行嵌套，但是这个操作其实什么都没做。

所以，以后的代码建议直接使用Tensor类进行操作，因为官方文档中已经将Variable设置成过期模块。

要想通过Tensor类本身就支持了使用autograd功能，只需要设置.requires_grad=True

Variable类中的的grad和grad_fn属性已经整合进入了Tensor类中

## Autograd
在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.5466, 0.4557, 0.8138, 0.7021, 0.3837],
        [0.3876, 0.5698, 0.0593, 0.0470, 0.1703],
        [0.6517, 0.8158, 0.8245, 0.9850, 0.4737],
        [0.6552, 0.1595, 0.8717, 0.7866, 0.3089],
        [0.5084, 0.7214, 0.0033, 0.5265, 0.7606]], requires_grad=True)

In [3]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.7107, 0.4346, 0.3709, 0.4259, 0.7132],
        [0.9172, 0.2533, 0.6769, 0.2052, 0.2006],
        [0.5626, 0.9939, 0.7434, 0.7676, 0.5683],
        [0.5391, 0.3264, 0.1772, 0.2826, 0.6309],
        [0.6488, 0.8614, 0.1256, 0.7032, 0.4141]], requires_grad=True)

In [4]:
z = torch.sum(x + y)
z

tensor(26.4426, grad_fn=<SumBackward0>)

在张量进行操作后，grad_fn已经被赋予了一个新的函数，这个函数引用了一个创建了这个Tensor类的Function对象。 Tensor和Function互相连接生成了一个非循环图，它记录并且编码了完整的计算历史。每个张量都有一个.grad_fn属性，如果这个张量是用户手动创建的那么这个张量的grad_fn是None。

下面我们来调用反向传播函数，计算其梯度

## 简单的自动求导

In [5]:
z.backward()
print(x.grad, y.grad)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


如果Tensor类表示的是一个标量（即它包含一个元素的张量），则不需要为backward()指定任何参数，但是如果它有更多的元素，则需要指定一个gradient参数，它是形状匹配的张量。 以上的 z.backward()相当于是z.backward(torch.tensor(1.))的简写。 这种参数常出现在图像分类中的单标签分类，输出一个标量代表图像的标签。

## 复杂的自动求导

In [6]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
z

tensor([[0.7150, 0.5451, 0.7948, 0.9113, 0.6791],
        [0.5611, 0.5545, 0.1474, 0.7216, 0.2016],
        [1.3377, 0.6702, 0.7980, 1.2841, 1.0018],
        [1.1722, 0.4004, 0.5327, 0.6950, 0.0217],
        [0.5750, 1.1271, 0.0825, 0.5925, 1.0580]], grad_fn=<AddBackward0>)

In [7]:
#我们的返回值不是一个标量，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like函数根据x生成一个张量
z.backward(torch.ones_like(x))
print(x.grad)

tensor([[0.7130, 0.9340, 0.5964, 1.5559, 0.4297],
        [1.4818, 0.3395, 0.7494, 1.4430, 0.8968],
        [1.7802, 1.5119, 1.6779, 1.6170, 1.9863],
        [1.0223, 0.7314, 0.1781, 1.6490, 0.2947],
        [1.3259, 1.0751, 0.1247, 0.9223, 1.9027]])


我们可以使用with torch.no_grad()上下文管理器临时禁止对已设置requires_grad=True的张量进行自动求导。这个方法在测试集计算准确率的时候会经常用到，例如：

In [8]:
with torch.no_grad():
    print((x + y*2).requires_grad)

False


使用.no_grad()进行嵌套后，代码不会跟踪历史记录，也就是说保存的这部分记录会减少内存的使用量并且会加快少许的运算速度。

## Autograd 过程解析
为了说明Pytorch的自动求导原理，我们来尝试分析一下PyTorch的源代码，虽然Pytorch的 Tensor和 TensorBase都是使用CPP来实现的，但是可以使用一些Python的一些方法查看这些对象在Python的属性和状态。 Python的 dir() 返回参数的属性、方法列表。z是一个Tensor变量，看看里面有哪些成员变量。

In [9]:
dir(z)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed_

返回很多，我们直接排除掉一些Python中特殊方法（以__开头和结束的）和私有方法（以_开头的，直接看几个比较主要的属性： .is_leaf：记录是否是叶子节点。通过这个属性来确定这个变量的类型 在官方文档中所说的“graph leaves”，“leaf variables”，都是指像x，y这样的手动创建的、而非运算得到的变量，这些变量称为创建变量。 像z这样的，是通过计算后得到的结果称为结果变量。

一个变量是创建变量还是结果变量是通过.is_leaf来获取的。

In [10]:
print("x.is_leaf="+str(x.is_leaf))
print("z.is_leaf="+str(z.is_leaf))

x.is_leaf=True
z.is_leaf=False


x是手动创建的没有通过计算，所以他被认为是一个叶子节点也就是一个创建变量，而z是通过x与y的一系列计算得到的，所以不是叶子结点也就是结果变量。

为什么我们执行z.backward()方法会更新x.grad和y.grad呢？ .grad_fn属性记录的就是这部分的操作，虽然.backward()方法也是CPP实现的，但是可以通过Python来进行简单的探索。

grad_fn：记录并且编码了完整的计算历史

In [11]:
z.grad_fn

grad_fn是一个AddBackward0类型的变量 AddBackward0这个类也是用Cpp来写的，但是我们从名字里就能够大概知道，他是加法(ADD)的反反向传播（Backward），看看里面有些什么东西

In [12]:
dir(z.grad_fn)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_register_hook_dict',
 '_saved_alpha',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'register_prehook',
 'requires_grad']

next_functions就是grad_fn的精华

In [13]:
z.grad_fn.next_functions

((<PowBackward0 at 0x11fd5ebb0>, 0), (<PowBackward0 at 0x11fd5e6a0>, 0))

In [14]:
xg = z.grad_fn.next_functions[0][0]
dir(xg)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_raw_saved_self',
 '_register_hook_dict',
 '_saved_exponent',
 '_saved_self',
 'metadata',
 'name',
 'next_functions',
 'register_hook',
 'register_prehook',
 'requires_grad']

In [15]:
x_leaf=xg.next_functions[0][0]
x_leaf
#type(x_leaf)

在PyTorch的反向图计算中，AccumulateGrad类型代表的就是叶子节点类型，也就是计算图终止节点。AccumulateGrad类中有一个.variable属性指向叶子节点。

In [16]:
x_leaf.variable

tensor([[0.3565, 0.4670, 0.2982, 0.7779, 0.2149],
        [0.7409, 0.1698, 0.3747, 0.7215, 0.4484],
        [0.8901, 0.7559, 0.8389, 0.8085, 0.9932],
        [0.5111, 0.3657, 0.0891, 0.8245, 0.1474],
        [0.6629, 0.5375, 0.0624, 0.4612, 0.9513]], requires_grad=True)

这个.variable的属性就是我们的生成的变量x

In [17]:
print("x_leaf.variable的id:"+str(id(x_leaf.variable)))
print("x的id:"+str(id(x)))

x_leaf.variable的id:4461131952
x的id:4461131952


In [18]:
assert(id(x_leaf.variable)==id(x))

这样整个规程就很清晰了：

当我们执行z.backward()的时候。这个操作将调用z里面的grad_fn这个属性，执行求导的操作。
这个操作将遍历grad_fn的next_functions，然后分别取出里面的Function（AccumulateGrad），执行求导操作。这部分是一个递归的过程直到最后类型为叶子节点。
计算出结果以后，将结果保存到他们对应的variable 这个变量所引用的对象（x和y）的 grad这个属性里面。
求导结束。所有的叶节点的grad变量都得到了相应的更新
最终当我们执行完c.backward()之后，a和b里面的grad值就得到了更新。

## 扩展Autograd
如果需要自定义autograd扩展新的功能，就需要扩展Function类。因为Function使用autograd来计算结果和梯度，并对操作历史进行编码。 在Function类中最主要的方法就是forward()和backward()他们分别代表了前向传播和反向传播。

一个自定义的Function需要一下三个方法：

`__init__`(optional)：如果这个操作需要额外的参数则需要定义这个Function的构造函数，不需要的话可以忽略。

forward()：执行前向传播的计算代码

backward()：反向传播时梯度计算的代码。 参数的个数和forward返回值的个数一样，每个参数代表传回到此操作的梯度。